# Generacion de datos

Creacion de carpetas y Definicion de funciones utilizadas

In [7]:
import numpy as np
import random
import string
import sys
from PIL import Image, ImageDraw, ImageFont
import os
from os import listdir, makedirs

# fuente (bistream vera) https://www.dafont.com/bitstream-vera-sans.font
font = ImageFont.truetype(r'Vera.ttf', 
                          size=55)

# ruta de guardado de imagenes
path = "../dataset/"

#declaracion alfabeto
alphabet = string.digits + string.ascii_letters
#Declaracion del alfabeto reconozible por el programa
recognizer_alphabet = ''.join(sorted(set(alphabet.upper())))

#creacion de carpetas de clasificacion
for i in recognizer_alphabet:
    if not os.path.isdir(path + "data_train/images/" + i):
        makedirs(path + "data_train/images/" + i)
    if not os.path.isdir(path + "data_train/clasification/" + i):
        makedirs(path + "data_train/clasification/" + i)
    if not os.path.isdir(path + "data_test/images/" + i):
        makedirs(path + "data_test/images/" + i)
    if not os.path.isdir(path + "data_test/clasification/" + i):
        makedirs(path + "data_test/clasification/" + i)
    if not os.path.isdir(path + "data_valid"):
        makedirs(path + "data_valid")
#    if not os.path.isdir(path + "data_valid/clasification/" + i):
#        makedirs(path + "data_valid/clasification/" + i)

#retorno de una posicion aleatoria
def rndPointDisposition(dx, dy):
    x = int(random.uniform(-dx, dx))
    y = int(random.uniform(-dy, dy))
    return (x, y)

#retorno de 4 posiciones, area de deformacion
def quadPoints(size, disp1, disp2):
    w, h = size
    x1, y1 = disp1
    x2, y2 = disp2

    return (
        x1,    -y1,
        -x1,    h + y2,
        w + x2, h - y2,
        w - x2, y1)
    
#deforma la imagen de forma aleatoria
def rndLineTransform(image):
    w, h = image.size

    # default: 0.3 0.5
    dx = w * random.uniform(0.2, 0.4)
    dy = h * random.uniform(0.2, 0.4)

    #devuelve puntos aleatorios
    x1, y1 = [abs(z) for z in rndPointDisposition(dx, dy)]
    x2, y2 = [abs(z) for z in rndPointDisposition(dx, dy)]

    #devuelve el area de deformacion de la imagen
    quad = quadPoints((w, h), (x1, y1), (x2, y2))
    return image.transform(image.size, Image.QUAD,
                            data=quad, resample=Image.BICUBIC, fill=1)


#crea una nueva imagen deformada con fondo blanco
def deform_image(image):
    transformed_image = rndLineTransform(image)

    #crea imagen RGGA con dimesiones estandar y fondo blanco
    #se le combina con la imagen deformada para generar la imagen final
    new_image = Image.new('RGBA', (190, 80), color=(255, 255, 255))
    new_image.paste(transformed_image, transformed_image)

    return new_image

#dibuja una puntos
def draw_cross(ctx, x, y):
    ctx.point((x, y), 'black')
    ctx.point((x+1, y), 'black')
    ctx.point((x-1, y), 'black')
    ctx.point((x, y+1), 'black')
    ctx.point((x, y-1), 'black')

#dibuja una puntos aleatoria dentro de la imagen
def draw_random_cross(ctx):
    x1 = random.randint(1, 189)
    y1 = random.randint(1, 79)

    draw_cross(ctx, x1, y1)

#dibuja una linea dentro de la imagen
def draw_random_line(ctx):
    x1 = random.randint(0, 190)
    y1 = random.randint(0, 80)

    x2 = random.randint(0, 190)
    y2 = random.randint(0, 80)
    ctx.line((x1, y1, x2, y2), 'black')


#dibuja lineas y puntos de forma aleatoria
def draw_random_stuff(ctx):
    num_crosses = random.randint(80, 90)

    for i in range(num_crosses):
        draw_random_cross(ctx)   
    
    num_lines = random.randint(5, 7)

    for i in range(num_lines):
        draw_random_line(ctx)

#genera un captcha
def gen_captcha(text):
    #se crea una imagen de 190x80
    image = Image.new('RGBA', (190, 80), color=(255, 255, 255))
    draw = ImageDraw.Draw(image)

    #dibuja puntos y líneas al azar sin deformación y el texto
    draw_random_stuff(draw)
    draw.text((10, 5), text[0], fill='black', font=font,align='center')
    draw.text((55, 5), text[1], fill='black', font=font,align='center')
    draw.text((95, 5), text[2], fill='black', font=font,align='center')
    draw.text((135, 5), text[3], fill='black', font=font,align='center')
    del draw

    # passo 2: transforma a imagem
    image = deform_image(image)

    # passo 3: repetir passo 1
    draw = ImageDraw.Draw(image)
    draw_random_stuff(draw)
    del draw

    return image

#genera un texto de 4 caracteres
def gen_string(size=4, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

Creacion de captchas 

In [8]:


#se generaran datos imagenes de entrenamiento, testeo y validacion
num1 = 1000
num2= 150
num3 = 100

#Se generan por cada Caracter el numero num1,num2,num3 segun corresponda, ademas se verifica
#Si ya existe el numero de captchas objetivo, es decir si se quieren 10 cptchas de cada imagen 
#no se generaran mas de los ya existentes 


for i in range (2):
    if i == 0:
        num=num1
        tipo = "entrenamiento"
        path_aux = "data_train/images/"
    else:
        num=num2
        tipo = "testeo"
        path_aux = "data_test/images/"
        
    print(f"generando {num} captchas de {tipo} de cada caracter")
    #for que itera por todas los caracteres reconocibles del programa
    for character in recognizer_alphabet:
        #verifico cuantos archivos captcha existen en la ruta
        num_captcha = len(listdir(path + path_aux + character))
        #si este numero es menor entra para generar los archivos faltantes
        if num > num_captcha:
            print("creando caracter:", character)
            while num_captcha < num:
                #text = gen_string()
                image = gen_captcha(character*4)
                filename = str(num_captcha) + "_" + character*4 + ".png" 
                image.save(path + path_aux + character + "/" + filename)
                num_captcha += 1
    print(f"Ya existen los archivos necesarios de {tipo}\n")



print(f"generando {num3} captchas de validacion")
num_captcha = len(listdir(path + "data_valid"))
if num3 > num_captcha:
    while num_captcha < num3:
        text = gen_string()
        image = gen_captcha(text)
        filename = "x_" + text + ".png" 
        #creo una carpeta para el captcha, dentro de ella tambien guardo una carpeta de clasificacion
        makedirs(path + "data_valid/" + text + "/clasification/")
        j=0
        for i in text:
            path_clasification_apart=path + "data_valid/" + text + "/clasification/" +str(j)+'_'+ i
            if not os.path.isdir(path_clasification_apart):
                makedirs(path_clasification_apart)
                j=j+1
        image.save(path + "data_valid/"+ text + "/" +filename)
        num_captcha += 1
print("Ya existen los archivos necesarios de validacion")
        
print("\n\nlisto!")

generando 1000 captchas de entrenamiento de cada caracter
Ya existen los archivos necesarios de entrenamiento

generando 150 captchas de testeo de cada caracter
Ya existen los archivos necesarios de testeo

generando 100 captchas de validacion
Ya existen los archivos necesarios de validacion


listo!
